In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame

In [2]:
import sys
sys.path.append('./inc/')
import ID3
import C4_5

/Library/Python/2.7/site-packages/numpy/lib/arraysetops.py:216: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


             Pclass  Sex  Age  SibSp  Parch  Fare  Embarked
PassengerId                                                
1                 2    1   28      1      0    18         3
2                 0    0   51      1      0   207         1
3                 2    0   34      0      0    41         3
4                 0    0   47      1      0   189         3
5                 2    1   47      0      0    43         3
6                 2    1  110      0      0    51         2
7                 0    1   69      0      0   186         3
8                 2    1    6      3      1   124         3
9                 2    0   35      0      2    74         3
10                1    0   18      1      0   154         1
train accuracy = 0.98736
test accuracy = 0.76536
             Pclass     Sex   Age  SibSp  Parch     Fare Embarked
PassengerId                                                      
1                 3    male  22.0      1      0   7.2500        S
2                 1  female  38.0

## Import the data set

In [3]:
Columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country','income-level']


In [4]:
raw_data = pd.read_csv('./census data/adult.data', names=Columns)
raw_data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income-level
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


In [4]:
raw_data = pd.read_csv('./census data/adult.data', names=Columns)
raw_data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income-level
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


In [137]:
def id3_preprocess(raw_data, features, drop_feature, num_data):
    
    drop_feature += ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week', 'fnlwgt', 'income-level']
    
    X = raw_data.drop(drop_feature,1)[:num_data]
    y = raw_data['income-level'][:num_data]
    
    return X.as_matrix(), y.as_matrix(), features

In [147]:
def c45_preprocess(raw_data, features, drop_feature, num_data):
    drop_feature += ['fnlwgt', 'income-level']
    X = raw_data.drop(drop_feature,1)[:num_data]
    y = raw_data['income-level'][:num_data]
    
    print X.head()
    #print y
    
    for feature in drop_feature:
        features.remove(feature)
    
    con_set = set()
    con_list = ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
    for i in con_list:
        if i in features:
            print i
            con_set.add(features.index(i))
            print features.index(i)
    
    print con_set
    return X.as_matrix(),y.as_matrix(),features, con_set

In [148]:
def training(raw_data, features, drop_feature, num_data,version,filename):
    
    if version == 'c45':
        X_train, y_train, feature_list, con_set = c45_preprocess(raw_data, features, drop_feature, num_data)
        
        root = C4_5.C45(continuous=con_set, max_depth=2)
        root.fit(X_train, y_train)
        
        result_list = ['<=50K', '>50K']
        
        C4_5.draw(root,feature_list, result_list, filename)
        
    elif version == 'id3':
        
        X_train, y_train, feature_list = id3_preprocess(raw_data, features, drop_feature, num_data)
        
        root = ID3.ID3(max_depth=2)
        root.fit(X_train, y_train)
        
        result_list = ['<=50K', '>50K']
        
        ID3.draw(root,feature_list, result_list, filename)
        
    else:
        print "version can only be either c45 or id3"

## Implementing ID3

In [149]:
features = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country','income-level']
drop_feature = []
num_data = 1000
training(raw_data, features, drop_feature, num_data, version='id3', filename = './id3_census_training.png')

Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large


## Implementing C4.5

In [150]:
features = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country','income-level']
drop_feature = []
num_data = 10000

training(raw_data, features, drop_feature, num_data, version = 'c45', filename = './c45_census_training.png')

   age          workclass   education  education-num       marital-status  \
0   39          State-gov   Bachelors             13        Never-married   
1   50   Self-emp-not-inc   Bachelors             13   Married-civ-spouse   
2   38            Private     HS-grad              9             Divorced   
3   53            Private        11th              7   Married-civ-spouse   
4   28            Private   Bachelors             13   Married-civ-spouse   

           occupation    relationship    race      sex  capital-gain  \
0        Adm-clerical   Not-in-family   White     Male          2174   
1     Exec-managerial         Husband   White     Male             0   
2   Handlers-cleaners   Not-in-family   White     Male             0   
3   Handlers-cleaners         Husband   Black     Male             0   
4      Prof-specialty            Wife   Black   Female             0   

   capital-loss  hours-per-week  native-country  
0             0              40   United-States  
1   

In [36]:
root = C4_5.C45(continuous={0,3,9,10,11},max_depth=2)

In [37]:
root.fit(X_train,y_train)

In [38]:
feature_list =( ['age', 'workclass', 'education', 'education-num', 'marital-status',         
                'occupation', 'relationship','race', 'sex capital-gain', 'capital-loss', 'hours-per-week', 'native-country'] )
result_list = ['<=50K', '>50K']

In [39]:
C4_5.draw(root,feature_list, result_list, './c45_census.png')

In [40]:
from sklearn.model_selection import train_test_split

X_train2, X_test, y_train2, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

clf = C4_5.C45(continuous={0,3,9,10,11})
clf.fit(X_train2, y_train2)
clf.score(X_test, y_test)

0.8

In [41]:
X_test.shape

(2000, 13)

# Functionalized Training Procedure

### Using subfeatures for training

#### education and occupation are stronly related, so we are going to use only one of them.

##### implementing ID3

##### Without education

In [121]:
features = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country','income-level']
drop_feature = ['education']
num_data = 1000
training(raw_data, features, drop_feature, num_data, version='id3', filename = './id3_census_without_edu.png')

           workclass       marital-status          occupation    relationship  \
0          State-gov        Never-married        Adm-clerical   Not-in-family   
1   Self-emp-not-inc   Married-civ-spouse     Exec-managerial         Husband   
2            Private             Divorced   Handlers-cleaners   Not-in-family   
3            Private   Married-civ-spouse   Handlers-cleaners         Husband   
4            Private   Married-civ-spouse      Prof-specialty            Wife   

     race      sex  native-country  
0   White     Male   United-States  
1   White     Male   United-States  
2   White     Male   United-States  
3   Black     Male   United-States  
4   Black   Female            Cuba  
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Dep

##### Without occupation

In [122]:
features = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country','income-level']
drop_feature = ['occupation']
num_data = 1000
training(raw_data, features, drop_feature, num_data, version='id3', filename = './id3_census_without_occ.png')

           workclass   education       marital-status    relationship    race  \
0          State-gov   Bachelors        Never-married   Not-in-family   White   
1   Self-emp-not-inc   Bachelors   Married-civ-spouse         Husband   White   
2            Private     HS-grad             Divorced   Not-in-family   White   
3            Private        11th   Married-civ-spouse         Husband   Black   
4            Private   Bachelors   Married-civ-spouse            Wife   Black   

       sex  native-country  
0     Male   United-States  
1     Male   United-States  
2     Male   United-States  
3     Male   United-States  
4   Female            Cuba  
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Depth too large
Dep

##### impleenting C4.5

##### Without education

In [123]:
features = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country','income-level']
drop_feature = ['education']
num_data = 10000

training(raw_data, features, drop_feature, num_data, version = 'c45', filename = './c45_census_without_edu.png')

   age          workclass  education-num       marital-status  \
0   39          State-gov             13        Never-married   
1   50   Self-emp-not-inc             13   Married-civ-spouse   
2   38            Private              9             Divorced   
3   53            Private              7   Married-civ-spouse   
4   28            Private             13   Married-civ-spouse   

           occupation    relationship    race      sex  capital-gain  \
0        Adm-clerical   Not-in-family   White     Male          2174   
1     Exec-managerial         Husband   White     Male             0   
2   Handlers-cleaners   Not-in-family   White     Male             0   
3   Handlers-cleaners         Husband   Black     Male             0   
4      Prof-specialty            Wife   Black   Female             0   

   capital-loss  hours-per-week  native-country  
0             0              40   United-States  
1             0              13   United-States  
2             0           

##### Without occupation

In [124]:
features = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country','income-level']
drop_feature = ['occupation']
num_data = 10000

training(raw_data, features, drop_feature, num_data, version = 'c45', filename = './c45_census_without_occ.png')

   age          workclass   education  education-num       marital-status  \
0   39          State-gov   Bachelors             13        Never-married   
1   50   Self-emp-not-inc   Bachelors             13   Married-civ-spouse   
2   38            Private     HS-grad              9             Divorced   
3   53            Private        11th              7   Married-civ-spouse   
4   28            Private   Bachelors             13   Married-civ-spouse   

     relationship    race      sex  capital-gain  capital-loss  \
0   Not-in-family   White     Male          2174             0   
1         Husband   White     Male             0             0   
2   Not-in-family   White     Male             0             0   
3         Husband   Black     Male             0             0   
4            Wife   Black   Female             0             0   

   hours-per-week  native-country  
0              40   United-States  
1              13   United-States  
2              40   United-State

### Using merged features for training

#### under high school
11th, 9th, 7th-8th, 12th, 1st-4th, 10th, 5th-6th are merged together.
#### working for gov
Federal-gov, Local-gov, State-gov are merged together.
